In [4]:
import json
import pandas as pd
from pathlib import Path
from typing import Dict, Any

# 指定目录和 json 文件
data_dir = Path('.')
json_files = [
    'rebuttal_set_fl.json',
    'refactored_alpha_gfn_set_new_icml.json',
    # 'rebuttal_set_temp_old.json',
    'refactored_alpha_gfn_set_new_icml_fl0.json'
]

def flatten_dict(d: Dict[str, Any], parent_key: str = '', sep: str = '_') -> Dict[str, Any]:
    """递归展平嵌套字典"""
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# 加载所有 json 文件并解析为 DataFrame
all_data = []

for file in json_files:
    file_path = data_dir / file
    with open(file_path, 'r') as f:
        data = json.load(f)

    # 每个 json 文件的顶层 key 是实验名称，value 是参数字典
    for exp_name, exp_params in data.items():
        flat_params = flatten_dict(exp_params)
        flat_params['experiment_name'] = exp_name  # 保存实验名称
        flat_params['source_file'] = file  # 保存来源文件
        all_data.append(flat_params)

# 转换为 DataFrame
df = pd.DataFrame(all_data)

print(f"✓ 加载完成！共 {len(df)} 行")
print(f"✓ 列数: {len(df.columns)}")
print(f"\n数据源: {json_files}")
print(f"\nDataFrame 形状: {df.shape}")
print(f"\n前几行:")
df.head()

✓ 加载完成！共 360 行
✓ 列数: 115

数据源: ['rebuttal_set_fl.json', 'refactored_alpha_gfn_set_new_icml.json', 'refactored_alpha_gfn_set_new_icml_fl0.json']

DataFrame 形状: (360, 115)

前几行:


,_runtime,_step,_timestamp,action_dim,alpha,alpha_init,alpha_logit,alpha_lr,alpha_sched,alpha_train_first_steps,...,use_grad_clip,use_modified_reward,use_replay_buffer,wdb,experiment_name,source_file,ckpt_every,ckpt_save_history,ckpt_save_test_set,resume
0,205719.880689,200,1.763940e+09,80,0.503678,0.7,0.014711,0.001,hold_exp,False,...,False,False,False,True,ss(0)_ms(0)_md(on)_mdfr(0)_upb(1)_sz(medium)_m...,rebuttal_set_fl.json,NaN,NaN,NaN,NaN
1,19711.694562,200,1.763543e+09,30,0.501839,0.6,0.007356,0.001,hold_exp,False,...,False,False,False,True,ss(4)_ms(0)_md(on)_mdfr(0)_upb(1)_sz(small)_m(...,rebuttal_set_fl.json,NaN,NaN,NaN,NaN
2,27512.494409,200,1.763515e+09,30,0.494483,0.2,-0.022068,0.001,hold_exp,False,...,False,False,False,True,ss(4)_ms(0)_md(on)_mdfr(0)_upb(1)_sz(small)_m(...,rebuttal_set_fl.json,NaN,NaN,NaN,NaN
3,154038.657995,200,1.763714e+09,80,0.501839,0.6,0.007356,0.001,hold_exp,False,...,False,False,False,True,ss(0)_ms(0)_md(on)_mdfr(0)_upb(1)_sz(medium)_m...,rebuttal_set_fl.json,NaN,NaN,NaN,NaN
4,22856.400692,200,1.763469e+09,30,0.507356,0.9,0.029425,0.001,hold_exp,False,...,False,False,False,True,ss(4)_ms(0)_md(on)_mdfr(0)_upb(1)_sz(small)_m(...,rebuttal_set_fl.json,NaN,NaN,NaN,NaN


In [5]:
import re
import numpy as np

# 复制一份以免污染原始 df
df_all = df.copy()

# 解析实验名称中的参数
def parse_exp_name(exp_name: str) -> dict:
    params = {}
    ss_match = re.search(r'ss\((\d+)\)', exp_name)
    if ss_match:
        params['seed'] = int(ss_match.group(1))

    m_match = re.search(r'_m\(([^)]+)\)', exp_name)
    if m_match:
        params['method'] = m_match.group(1)

    # 提取size sz(xxx)
    sz_match = re.search(r'_sz\(([^)]+)\)', exp_name)
    if sz_match:
        params['size'] = sz_match.group(1)

    a_match = re.search(r'_a\(([^)]+)\)', exp_name)
    if a_match:
        try:
            params['alpha_init_from_name'] = float(a_match.group(1))
        except ValueError:
            params['alpha_init_from_name'] = np.nan

    return params

parsed_rows = []
for _, row in df_all.iterrows():
    name = row.get('experiment_name', '')
    parsed = parse_exp_name(name)
    method = parsed.get('method')
    
    # 来自这个文件的method加上"fl-"前缀
    source_file = row.get('source_file', '')
    if source_file in ['refactored_alpha_gfn_set_new_icml.json']:
        if method:
            method = f"fl_{method}"
    
    parsed_rows.append({
        **row,
        'seed': parsed.get('seed'),
        'method': method,
        'size': parsed.get('size'),
        'alpha_init_name': parsed.get('alpha_init_from_name'),
        'alpha': row.get('alpha_init', row.get('alpha'))
    })

df_all = pd.DataFrame(parsed_rows)

# 过滤掉来自 rebuttal_set_fl.json 且 method 为 subtb_gfn 的数据
before_filter = len(df_all)
df_all = df_all[~((df_all['source_file'] == 'rebuttal_set_fl.json') & (df_all['method'] == 'subtb_gfn'))]
after_filter = len(df_all)
if before_filter > after_filter:
    print(f"✓ 已过滤掉 {before_filter - after_filter} 条数据（来自 rebuttal_set_fl.json 且 method=subtb_gfn）\n")

print(f"总实验数: {len(df_all)}")
print(f"Unique seeds: {sorted(df_all['seed'].dropna().unique())}")
print(f"Unique sizes: {sorted(df_all['size'].dropna().unique())}")
print(f"Unique alphas (from alpha_init): {sorted(df_all['alpha'].dropna().unique())}")
print(f"Unique objectives/methods: {sorted(df_all['method'].dropna().unique())}")
print("\n数据来源分布:")
print(df_all.groupby(['source_file', 'method', 'size']).size())

# 使用指定的评估指标
metrics = ['modes', 'mean_top_1000_R', 'mean_top_1000_similarity', 'spearman_corr_test']

# 检查哪些指标存在
available_metrics = [m for m in metrics if m in df_all.columns]
missing_metrics = [m for m in metrics if m not in df_all.columns]

if missing_metrics:
    print(f"⚠️ 缺失指标: {missing_metrics}")
    print(f"✓ 可用指标: {available_metrics}\n")
    metrics = available_metrics

objectives = sorted(df_all['method'].dropna().unique())
sizes = sorted(df_all['size'].dropna().unique())

print(f"\n{'=' * 80}")
print(f"找到 {len(objectives)} 个 objectives: {objectives}")
print(f"找到 {len(sizes)} 个 sizes: {sizes}")
print(f"{'=' * 80}")

# 按 method, size, alpha 分组统计（保留所有 alpha）
print("\n" + "=" * 80)
print("按 method, size, alpha 分组统计（保留所有 alpha）")
print("=" * 80)

grouped_data_records = []
for obj in objectives:
    for size in sizes:
        df_obj = df_all[(df_all['method'] == obj) & (df_all['size'] == size)].copy()
        if len(df_obj) == 0:
            continue

        print(f"\n{'=' * 80}")
        print(f"Objective: {obj}, Size: {size}")
        print(f"{'=' * 80}")
        print(f"  实验数: {len(df_obj)}")
        print(f"  来源文件: {df_obj['source_file'].unique()}")

        # 按 alpha 分组并计算均值和标准差
        for alpha in sorted(df_obj['alpha'].dropna().unique()):
            df_alpha = df_obj[df_obj['alpha'] == alpha]
            record = {
                'method': obj,
                'size': size,
                'alpha': alpha,
                'n_seeds': len(df_alpha)
            }
            for m in metrics:
                record[f'{m}_mean'] = df_alpha[m].mean()
                record[f'{m}_std'] = df_alpha[m].std()
            grouped_data_records.append(record)

        # 打印该组的统计信息
        df_group_summary = df_obj.groupby('alpha')[metrics].agg(['mean', 'std']).reset_index()
        print("\n按 alpha 分组的统计信息:")
        print(df_group_summary.to_string(index=False))

df_grouped = pd.DataFrame(grouped_data_records)
print("\n" + "=" * 80)
print("汇总数据（所有 alpha）")
print("=" * 80)
print(df_grouped.to_string(index=False))

# 保存数据
import os
os.makedirs('results', exist_ok=True)
output_path = Path('results') / 'data_all_alpha.csv'
df_grouped.to_csv(output_path, index=False)
print(f"\n✓ 已保存到: {output_path}")


✓ 已过滤掉 90 条数据（来自 rebuttal_set_fl.json 且 method=subtb_gfn）

总实验数: 270
Unique seeds: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)]
Unique sizes: ['large', 'medium', 'small']
Unique alphas (from alpha_init): [np.float64(0.1), np.float64(0.2), np.float64(0.3), np.float64(0.4), np.float64(0.5), np.float64(0.6), np.float64(0.7), np.float64(0.8), np.float64(0.9)]
Unique objectives/methods: ['fl_subtb_gfn', 'subtb_gfn']

数据来源分布:
source_file                                 method        size  
rebuttal_set_fl.json                        fl_subtb_gfn  medium    45
                                                          small     45
refactored_alpha_gfn_set_new_icml.json      fl_subtb_gfn  large     45
refactored_alpha_gfn_set_new_icml_fl0.json  subtb_gfn     large     45
                                                          medium    45
                                                          small     45
dtype: int64

找到 2 个 objectives: ['fl_subtb_gfn', 'subtb_gfn']
找